In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import trimesh
from skimage import measure
import meshplot as mp
from torch.utils.data import DataLoader, Dataset
import os
import time
from datetime import timedelta, datetime
import random
import math

### Load decoder and shapecode

In [3]:
def process_decoder_log(datetime):
    with open(os.path.join('./models/', datetime, f'autodecoder_{datetime}.log')) as f:
        f = f.readlines()

    for line in f:
        if "load_all_shapes" in line:
            load_all_shapes = line.split(' = ')[1].split(',')[0].strip()
        if "n_shapes" in line:
            n_shapes = line.split(' = ')[2].split(',')[0].strip()
        if load_all_shapes == 'True':
            n_shapes = 6778
        if "dropout" in line:
            dropout = line.split('=')[3].split(',')[0].strip()
        if "weight_norm" in line:
            weight_norm = line.split('=')[5].split(',')[0].strip()
        if "use_tanh" in line:
            use_tanh = line.split('=')[6].strip()
        if "n_points_to_load" in line:
            n_points_to_load = line.split(' = ')[2].split(',')[0].strip()
        if "loss" in line and 'Validation' not in line:
            epoch_trained = line.split('-')[2].split('loss')[0].strip()
    
    return int(n_shapes), dropout=="True", weight_norm=="True", use_tanh=="use_tanh", int(epoch_trained)

def get_decoder(datetime, decoder_epoch=None):
    n_shapes, dropout, weight_norm, use_tanh, epoch_trained = process_decoder_log(datetime)
    
    if decoder_epoch is None: 
        decoder_epoch = math.floor(epoch_trained/10) * 10 # use the max trained epoch if not specified

    from train_decoder import MLP
    decoder = MLP(n_shapes, 256, 512, dropout, 0.2, weight_norm, use_tanh)
    decoder.load_state_dict(torch.load(f'./models/{datetime}/autodecoder_{datetime}_{decoder_epoch}'))

    return decoder, decoder_epoch
    
def get_shapecode(datetime, shapecode_epoch=None, shape_indices=None, n_shapes=10, random_or_not=False):
    max_n_shapes, dropout, weight_norm, use_tanh, epoch_trained = process_decoder_log(datetime)
    
    if shapecode_epoch is None: 
        shapecode_epoch = math.floor(epoch_trained/10) * 10 # use the max trained epoch if not specified

    shapecode = torch.nn.Embedding(max_n_shapes, 256)
    shapecode.load_state_dict(torch.load(f'./models/{datetime}/shapecode_{datetime}_{shapecode_epoch}'))
    shapecode=shapecode.weight.data.detach()
    # print(shapecode.shape)
    
    if shape_indices is None:
        if random_or_not:
            random.seed()
        else:
            random.seed(0)
        shape_indices = random.sample(range(max_n_shapes), n_shapes) # randomly sample n_shapes points out of max_n_shapes
    
    shapecode = shapecode[torch.tensor(shape_indices)]        
    
    return shapecode, shapecode_epoch, shape_indices


### Diffusion - Helper Functions

In [17]:
def make_beta_schedule(schedule='linear', n_timesteps=1000, start=1e-5, end=1e-2):
    if schedule == 'linear':
        betas = torch.linspace(start, end, n_timesteps)
    elif schedule == "quad":
        betas = torch.linspace(start ** 0.5, end ** 0.5, n_timesteps) ** 2
    elif schedule == "sigmoid":
        betas = torch.linspace(-6, 6, n_timesteps)
        betas = torch.sigmoid(betas) * (end - start) + start
    return betas

############ Gloabl variables ############
n_steps = 30000
betas = make_beta_schedule(schedule='linear', n_timesteps=n_steps, start=1e-5, end=1e-2)
alphas = 1 - betas
alphas_prod = torch.cumprod(alphas, 0)
alphas_prod_p = torch.cat([torch.tensor([1]).float(), alphas_prod[:-1]], 0)
alphas_bar_sqrt = torch.sqrt(alphas_prod)
print(alphas_bar_sqrt)
one_minus_alphas_bar_log = torch.log(1 - alphas_prod)
one_minus_alphas_bar_sqrt = torch.sqrt(1 - alphas_prod)


tensor([1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000])


In [18]:
def extract(input, t, x):
    shape = x.shape
    out = torch.gather(input, 0, t.to(input.device))
    reshape = [t.shape[0]] + [1] * (len(shape) - 1)
    return out.reshape(*reshape)

### Diffusion 

In [19]:
def q_sample(x_0, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_0)
    alphas_t = extract(alphas_bar_sqrt, t, x_0)
    alphas_1_m_t = extract(one_minus_alphas_bar_sqrt, t, x_0)
    return (alphas_t * x_0 + alphas_1_m_t * noise)

def p_sample(model, x, t):
    t = torch.tensor([t])
    # Factor to the model output
    eps_factor = ((1 - extract(alphas, t, x)) / extract(one_minus_alphas_bar_sqrt, t, x))
    # Model output
    eps_theta = model(x, t)
    # Final values
    mean = (1 / extract(alphas, t, x).sqrt()) * (x - (eps_factor * eps_theta))
    # Generate z
    z = torch.randn_like(x)
    # Fixed sigma
    sigma_t = extract(betas, t, x).sqrt()
    sample = mean + sigma_t * z
    return (sample)


### Plotting Helper Functions

In [92]:
x = np.linspace(-1, 1, 100, dtype=np.float32)
y = np.linspace(-1, 1, 100, dtype=np.float32)
z = np.linspace(-1, 1, 100, dtype=np.float32)
P = np.vstack(np.meshgrid(x,y,z)).reshape(3,-1).T  # format: [[x1, y1, z1], [x1, y1, z2], [] ...]
P = torch.from_numpy(P)
P = torch.chunk(P, 100)

In [93]:
def reconstruct_shapes(decoder, shapecode):
    sdfs = []
    for points in P:
        points = points.view(-1, 3)
        # sdfs.append(decoder(torch.cat((points, (std*shapecode+mean).repeat(points.shape[0], 1)), dim=1)).detach().numpy())
        sdfs.append(decoder(torch.cat((points, (shapecode).repeat(points.shape[0], 1)), dim=1)).detach().numpy())
    sdfs = np.concatenate(sdfs, axis=0)
    sdfs = np.resize(sdfs, (len(x), len(y), len(z))) 
    verts, faces, normals, values = measure.marching_cubes(sdfs, 0)
    return verts, faces

### Compare denoised sdfs & original decoder sdfs

#### 0. Run denoising OR load saved denoised shape 

In [26]:
# TODO: gracefully clean up dependencies including q_sample(), p_sample(), get_decoder(), get_shapecode()
def run_denoising(diffusion_model_dt, 
                  diffusion_model_idx, 
                  ref_shape_idx=94, # shape index to add noise and denoise (caution:  note that shape_idx != shape_indices_from_decoder[shape_idx])
                  timestep = (30000-1), # pick timestep to add noise
                  print_original=False, 
                  print_noised=False, 
                  print_denoised=True):

    # load diffusion model
    model_filename = f'./diffusion logs & models/{diffusion_model_dt}/conditional model_{diffusion_model_dt}_{diffusion_model_idx}'
    from train_diffusion import ConditionalModel
    model = ConditionalModel(30000)
    model.load_state_dict(torch.load(model_filename))
    model.eval()

    
    # load diffusion log and the number of shpaes used for diffusion (n_shapes) and the the decoder model used (decoder_model)
    log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'
    with open(log_filename) as f:
        f = f.readlines()
    for line in f: 
        if "# of shapes used for diffusion" in line :
            n_shapes = int(line.split('=')[1].strip())
        if "Decoder model used" in line :
            decoder_model = line.split('=')[1].strip()

            
    # load the decoder and shapecodes
    decoder, _ = get_decoder(decoder_model) # speicfy decoder_epoch if needed 
    shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_model, n_shapes=n_shapes) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed
    # TODO: assert that shape_indices from the diffusion log == shape_indices_from_decoder

    std = np.std(np.asarray(shapecode))
    mean = np.mean(np.asarray(shapecode))
    shapecode = (shapecode-mean)/std

    
    # prepare points to sample when reconstructing shapes using the decoder
    x = np.linspace(-1, 1, 100, dtype=np.float32)
    y = np.linspace(-1, 1, 100, dtype=np.float32)
    z = np.linspace(-1, 1, 100, dtype=np.float32)
    P = np.vstack(np.meshgrid(x,y,z)).reshape(3,-1).T  # format: [[x1, y1, z1], [x1, y1, z2], [] ...]
    P = torch.from_numpy(P)
    P = torch.chunk(P, 100)
    
    
    # retrieve original shapecode corresponding to the ref_shape_idx
    original_code=std*shapecode[ref_shape_idx]+mean
    
    if print_original: 
        print('Printing the original shape')
        sdfs = []
        for points in P:
            points = points.view(-1, 3)
            sdfs.append(decoder(torch.cat((points,  original_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
        sdfs = np.concatenate(sdfs, axis=0)
        sdfs = np.resize(sdfs, (len(x), len(y), len(z))) 
        verts, faces, normals, values = measure.marching_cubes(sdfs, 0)
        mp.plot(verts, faces)

    
    # add noise to the original shapecode 
    noised_code = std*q_sample(shapecode[ref_shape_idx], torch.tensor([timestep]))+mean

    if print_noised:
        print('Printing the noised shape')
        sdfs = []
        for points in P:
            points = points.view(-1, 3)
            sdfs.append(decoder(torch.cat((points,  noised_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
        sdfs = np.concatenate(sdfs, axis=0)
        sdfs = np.resize(sdfs, (len(x), len(y), len(z))) 
        verts, faces, normals, values = measure.marching_cubes(sdfs, 0)
        mp.plot(verts, faces)
    
    
    # denoise the noised code 
    denoised_code = noised_code
    for i in reversed(range(timestep)):
        denoised_code = p_sample(model, denoised_code, i)
    denoised_code = std*denoised_code+mean

    denoised_sdfs = []
    for points in P:
        points = points.view(-1, 3)
        denoised_sdfs.append(decoder(torch.cat((points,  denoised_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
    denoised_sdfs = np.concatenate(denoised_sdfs, axis=0)
    denoised_sdfs = np.resize(denoised_sdfs, (len(x), len(y), len(z))) 
    denoised_verts, faces, normals, values = measure.marching_cubes(denoised_sdfs, 0)
    
    if print_denoised: 
        print('Printing the denoised shape')
        mp.plot(denoised_verts, faces)

    
    # post-process the denoised verts (caution: 'model:09272022_165500_8200...' is saved without this post-processing)
    scale = np.array([x[1] - x[0], y[1] - y[0], z[1] - z[0]])
    offset = np.array([x[0], y[0], z[0]])
    denoised_verts = denoised_verts*scale + offset

    
    # save the denoise shape 
    denoising = datetime.now() 
    denoising = denoising.strftime("%m%d%Y_%H%M%S")

    if not os.path.isdir(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}'):
        os.mkdir(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}')
        print(f'created directory: ./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}')

    np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising}', denoised_code=denoised_code.detach().numpy(), verts=denoised_verts, faces=faces)
    
    return denoising

In [27]:
run_denoising('10062022_011812', 6100, print_original=True, print_noised=True, print_denoised=True)

Printing the original shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.263183…

Printing the noised shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.840095…

Printing the denoised shape


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.607093…

'10062022_212728'

In [95]:
# loading the saved denoised shape
data  = np.load('./denoising results/model:09272022_165500_8200,shape:94/09302022_082805.npz')  # change the address
denoised_verts = data['verts']
denoised_faces = data['faces']
mp.plot(denoised_verts, denoised_faces)
denoised_code = data['denoised_code'] # this is the already denormalized code (ie after oeprating std*denoised_code+mean)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.506714…

#### 1. cosine similarity (latent vector)

In [96]:
def compute_cosine_similarity(denoising_dt='09302022_082805'):
    # retrieve denoised code to be compared with the original shape codes
    diffusion_model_idx = 8200
    ref_shape_idx = 94
    diffusion_model_dt = '09272022_165500'
    # denoising_dt = '09302022_082805' 
    data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
    denoised_code = data['denoised_code'] # this is the already denormalized code (ie after oeprating std*denoised_code+mean)
    denoised_code = torch.tensor(denoised_code)

    # retrieve shape indices used for diffusion
    log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'

    with open(log_filename) as f:
        f = f.readlines()

    for idx, line in enumerate(f):
        if "Decoder model used" in line :
            if idx > 10:
                break
        if "Shape indices" in line:
            shape_indices = line.split('=')[1].strip().strip('[').strip(']')

    shape_indices = shape_indices.split(', ')
    shape_indices = [int(shape_index) for shape_index in shape_indices]

    # retrieve original shape codes
    decoder_dt = '09152022_190445'
    shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed
    assert shape_indices == shape_indices_from_decoder


    # compute cosine similarity between denoised shape code and original shape codes
    cos_sim = []
    cos = nn.CosineSimilarity(dim=0)
    for shape_index in shape_indices:
        cos_sim.append((cos((shapecode[shape_index]),torch.squeeze(denoised_code))).detach().numpy())
    # format: 
    # 3155 cos, 
    # 6209 cos

    # saving the computed results
    cos_sim = np.array(cos_sim)
    np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_cos_sim', cos_sim=cos_sim)
    
    return cos_sim

In [97]:
print(cos_sim)
for idx in (-cos_sim).argsort()[:10]:
    print(shape_indices[idx], cos_sim[idx])

[0.34347343 0.11811234 0.35731378 ... 0.16697209 0.24136283 0.35786706]
4173 0.63578224
6617 0.63213325
5380 0.62537885
5506 0.62325644
2382 0.61048734
1124 0.60014397
5009 0.59657407
687 0.5954612
3327 0.5945884
1442 0.59307635


In [99]:
decoder, _ = get_decoder(decoder_dt) 

for idx in (-cos_sim).argsort()[:10]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
    mp.plot(verts, faces)

ValueError: Surface level must be within volume data range.

#### 2. Euclidean distance (latent vector)

In [82]:
# retrieve denoised code to be compared with the original shape codes
diffusion_model_idx = 8200
ref_shape_idx = 94
diffusion_model_dt = '09272022_165500'
denoising_dt = '09302022_082805' # saving_dt
data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
denoised_code = data['denoised_code'] # this is the already denormalized code (ie after oeprating std*denoised_code+mean)
denoised_code = torch.tensor(denoised_code)

# retrieve shape indices used for diffusion
log_filename = f'./diffusion logs & models/{diffusion_model_dt}/{diffusion_model_dt}.log'

with open(log_filename) as f:
    f = f.readlines()

for idx, line in enumerate(f):
    if "Decoder model used" in line :
        if idx > 10:
            break
    if "Shape indices" in line:
        shape_indices = line.split('=')[1].strip().strip('[').strip(']')

shape_indices = shape_indices.split(', ')
shape_indices = [int(shape_index) for shape_index in shape_indices]

# retrieve original shape codes
decoder_dt = '09152022_190445'
shapecode, _, shape_indices_from_decoder = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed
assert shape_indices == shape_indices_from_decoder

std = np.std(np.asarray(shapecode))
mean = np.mean(np.asarray(shapecode))
shapecode = (shapecode-mean)/std

# compute cosine similarity between denoised shape code and original shape codes
euc_dist_diff = []
cos = nn.CosineSimilarity(dim=0)
for shape_index in shape_indices:
    euc_dist_diff.append(torch.linalg.norm((std*shapecode[int(shape_index)] + mean)-torch.squeeze(denoised_code)).detach().numpy())
    
# format: 
# 3155 cos, 
# 6209 cos
    
# saving the computed results
euc_dist_diff = np.array(euc_dist_diff)
np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_euc_dist_diff', euc_dist_diff=euc_dist_diff)

In [100]:
euc_dist_diff

print(euc_dist_diff)
for idx in euc_dist_diff.argsort()[:10]:
    print(shape_indices[idx], euc_dist_diff[idx])

[1.8496909 1.9283289 1.6956576 ... 2.0635808 1.71431   1.6427572]
3452 1.302804
6725 1.323305
5110 1.3270646
3704 1.3292283
1926 1.3684391
4560 1.3839875
4334 1.3858078
6755 1.3868244
6557 1.3869848
3342 1.3908254


In [101]:
decoder, _ = get_decoder(decoder_dt) 

for idx in euc_dist_diff.argsort()[:10]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
    mp.plot(verts, faces)

ValueError: Surface level must be within volume data range.

#### 3. chamfer distance & absolute sdfs 

In [102]:
from scipy.spatial import cKDTree as KDTree

def compute_symmetric_chamfer_distance(verts1, verts2):
    verts1_kd_tree = KDTree(verts1)
    one_distances, one_vertex_ids = verts1_kd_tree.query(verts2)

    verts2_kd_tree = KDTree(verts2)
    two_distances, two_vertex_ids = verts2_kd_tree.query(verts1)

    return np.mean(np.square(one_distances)) + np.mean(np.square(two_distances))

In [103]:
from scipy.spatial import cKDTree as KDTree

scale = np.array([x[1] - x[0], y[1] - y[0], z[1] - z[0]])
offset = np.array([x[0], y[0], z[0]])

symmetric_chamfers = []


# load denoised vertices to be compared
diffusion_model_dt = '09272022_165500'
diffusion_model_idx = 8200
ref_shape_idx = 94
denoising_dt = '09302022_082805' # saving_dt

data  = np.load(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}.npz')
denoised_verts = data['verts']
denoised_verts = denoised_verts*scale + offset


# load vertices for shapes reconstructed using the decoder model and original shape codes
# if vertices files do not exist, compute vertices for shapes reconstructed using the decoder model and original shape codes

decoder_dt = '09152022_190445'
if not os.path.isdir(f'./models/{decoder_dt}/reconstructed'):
    os.mkdir(f'./models/{decoder_dt}/reconstructed')
    print(f'created directory: ./models/{decoder_dt}/reconstructed')

decoder, _ = get_decoder(decoder_dt) 
shapecode, _, shape_indices = get_shapecode(decoder_dt, n_shapes=6778) # specify shapecode_epoch, shape_indices, n_shapes, random_or_not if needed

std = np.std(np.asarray(shapecode))
mean = np.mean(np.asarray(shapecode))
shapecode = (shapecode-mean)/std

    
for idx in range(len(shapecode)): # note that this idx != shape_indices[idx]
    
    if not os.path.exists(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}'):
        original_code=std*shapecode[idx]+mean

        sdfs = []
        for points in P:
            points = points.view(-1, 3)
            sdfs.append(decoder(torch.cat((points,  original_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
        sdfs = np.concatenate(sdfs, axis=0)
        sdfs = np.resize(sdfs,  (len(x), len(y), len(z))) 
        verts, faces, _, _ = measure.marching_cubes(sdfs, 0)
        verts = verts*scale + offset
        
        np.savez(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}', verts=verts, faces=faces)
    else:
        data = np.load(f'./models/{decoder_dt}/reconstructed/{shape_indices[idx]}')
        verts = data['verts']

    symmetric_chamfer = compute_symmetric_chamfer_distance(denoised_verts, verts)
    print(f'{shape_indices[idx]}: {symmetric_chamfer}')
    symmetric_chamfers.append(symmetric_chamfer)
    
    if (idx%10 == 0) or (idx == len(shapecode)-1):
        np.savez(f'./denoising results/model:{diffusion_model_dt}_{diffusion_model_idx},shape:{ref_shape_idx}/{denoising_dt}_chamfer', symmetric_chamfers=np.array(symmetric_chamfers))

3155: 0.03548387802587052
6209: 0.02883628414720759
3445: 0.0387960786590546
331: 0.021309713795765704
2121: 0.024491355441123026
4188: 0.07527068074778859
3980: 0.05195236335998118
3317: 0.01787335579616698
6420: 0.08188897530087884
2484: 0.034361564556328175
3904: 0.07061771776579168
2933: 0.02556465409677128


KeyboardInterrupt: 

In [104]:
data = np.load('./denoising results/model:09272022_165500_8200,shape:94/09302022_082805_chamfer.npz')
sym_chamf = data['symmetric_chamfers']
for idx in (sym_chamf).argsort()[:10]:
    print(shape_indices[idx], sym_chamf[idx])

3317 0.01787335579616698
331 0.021309713795765704
2121 0.024491355441123026
6209 0.02883628414720759
2484 0.034361564556328175
3155 0.03548387802587052
3445 0.0387960786590546
3980 0.05195236335998118
3904 0.07061771776579168
4188 0.07527068074778859


In [105]:
decoder, _ = get_decoder(decoder_dt) 

for idx in (sym_chamf).argsort()[:20]:
    verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
    mp.plot(verts, faces)

ValueError: Surface level must be within volume data range.

### Compare multiple denoising runs

In [85]:
for _ in range(10):
    denoising_dt = run_denoising()
    data  = np.load(f'./denoising results/model:09272022_165500_8200,shape:94/{denoising_dt}.npz')
    denoised_verts = data['verts']
    denoised_faces = data['faces']
    # print('denoised shape') 
    p = mp.plot(denoised_verts*scale+offset, denoised_faces)
    cos_sim = compute_cosine_similarity(denoising_dt)
    cos_sim = np.array(cos_sim)

    for i, idx in enumerate((-cos_sim).argsort()[:2]):
        verts, faces = reconstruct_shapes(decoder, shapecode[shape_indices[idx]])
        # print(f'shape most similar (cosine similarity)')
        p.add_mesh(verts*scale+offset+ np.tile([0, 0, i*2+2],(len(verts),1)), faces) #TODO: needs sacling

    p

94


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002417…

ValueError: Surface level must be within volume data range.

### Plot original shapes

In [ ]:
for shape_idx in range(len(shapecode)):

    original_code=std*shapecode[shape_idx]+mean

    sdfs = []
    for points in P:
        points = points.view(-1, 3)
        sdfs.append(decoder(torch.cat((points,  original_code.repeat(points.shape[0], 1)), dim=1)).detach().numpy())
    sdfs = np.concatenate(sdfs, axis=0)
    sdfs = np.resize(sdfs, (len(x), len(y), len(z))) 
    verts, faces, normals, values = measure.marching_cubes(sdfs, 0)
    mp.plot(verts, faces)
